<a href="https://colab.research.google.com/github/KimDaeUng/t2i_bert_finetuning/blob/master/BERT_fine_tuning_on_CUB_Captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT on CUB Captions (domain adaptation)

In [ ]:
!pip install transformers
!pip install nlp
!pip install pyarrow

     |████████████████████████████████| 890kB 9.3MB/s 
     |████████████████████████████████| 1.1MB 31.2MB/s 
     |████████████████████████████████| 3.0MB 54.2MB/s 
     |████████████████████████████████| 890kB 38.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4c4119721f0877e313e4be9e2891f2825bd0a33bfff9601586aa0b3001d08362
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.7MB 8.7MB/s 
     |████████████████████████████████| 17.3MB 147kB/s 
     |████████████████████████████████| 245kB 56.5MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
import os
from glob import glob

## 1. Prepare dataset

In [ ]:
# Mount google drive
from os import path
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Download the caption file
!wget "https://github.com/KimDaeUng/t2i_bert_finetuning/blob/master/text_c10.zip" -O '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10.zip'

In [ ]:
# Unzip the caption file
!unzip '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10.zip' -d '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10'

In [ ]:
# Get text file path list
root_text_path = "/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/*/*.txt"
text_list = glob(root_text_path)

In [ ]:
# Check the path list
text_list[:5]

['/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/185.Bohemian_Waxwing/Bohemian_Waxwing_0060_796642.txt',
 '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/185.Bohemian_Waxwing/Bohemian_Waxwing_0078_796649.txt',
 '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/185.Bohemian_Waxwing/Bohemian_Waxwing_0102_796692.txt',
 '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/185.Bohemian_Waxwing/Bohemian_Waxwing_0054_796623.txt',
 '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/185.Bohemian_Waxwing/Bohemian_Waxwing_0084_796639.txt']

In [ ]:
text_list.sort()

In [ ]:
len(text_list)

11788

In [ ]:
from nlp import load_dataset

In [ ]:
whole_dataset = load_dataset('text', data_files=text_list)

Using custom data configuration default


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6f7df17d102c4e7c/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b. Subsequent calls will reuse this data.


In [ ]:
# Check the path list
whole_dataset['train']['text'][:5]

['the medium sized bird has a dark grey color, a black downward curved beak, and long wings.\n',
 'the bird is dark grey brown with a thick curved bill and a flat shaped tail.\n',
 'bird has brown body feathers, white breast feathers and black beak\n',
 'this bird has a dark brown overall body color, with a small white patch around the base of the bill.\n',
 'the bird has very long and large brown wings, as well as a black body and a long black beak.\n']

In [ ]:
# Save dataset as single text file
path_lm_training = "/content/drive/My Drive/Colab Notebooks/t2i/data/CUB_whole_captions.txt"
with open(path_lm_training, 'w') as f:
    for item in whole_dataset['train']['text']:
        f.write(item)

100%|██████████| 117880/117880 [00:00<00:00, 1225145.34it/s]


In [ ]:
# Set BERT tokenizer
from transformers import BertTokenizer
path_model = "/content/drive/My Drive/Colab Notebooks/t2i/BERT_CUB"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, cache_dir=path_model)

In [ ]:
# Make encoded dataset 
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_lm_training,
    block_size=128,
)

CPU times: user 33.2 s, sys: 6.07 ms, total: 33.3 s
Wall time: 33.4 s


## 2. Train a language model from pretrained weights(domain adaptation)

In [ ]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=path_model,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:247: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 1.2871446533203126, 'learning_rate': 3.642779587404995e-05, 'epoch': 0.2714440825190011, 'step': 500}
{'loss': 1.1318653564453125, 'learning_rate': 2.2855591748099893e-05, 'epoch': 0.5428881650380022, 'step': 1000}
{'loss': 1.080734375, 'learning_rate': 9.283387622149838e-06, 'epoch': 0.8143322475570033, 'step': 1500}


CPU times: user 7min 6s, sys: 3min 10s, total: 10min 16s
Wall time: 10min 21s


TrainOutput(global_step=1842, training_loss=1.1457514229607764)

In [ ]:
trainer.save_model(path_model)

## 3. Check that LM actually trained

In [ ]:
tokenizer.special_tokens_map

{'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'unk_token': '[UNK]'}

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=path_model,
    tokenizer=tokenizer
)

In [ ]:
fill_mask("The bird has head with [MASK].")

[{'score': 0.26184365153312683,
  'sequence': '[CLS] the bird has head with black. [SEP]',
  'token': 2304,
  'token_str': 'black'},
 {'score': 0.1308348923921585,
  'sequence': '[CLS] the bird has head with white. [SEP]',
  'token': 2317,
  'token_str': 'white'},
 {'score': 0.058785539120435715,
  'sequence': '[CLS] the bird has head with red. [SEP]',
  'token': 2417,
  'token_str': 'red'},
 {'score': 0.05665842443704605,
  'sequence': '[CLS] the bird has head with brown. [SEP]',
  'token': 2829,
  'token_str': 'brown'},
 {'score': 0.05123694986104965,
  'sequence': '[CLS] the bird has head with grey. [SEP]',
  'token': 4462,
  'token_str': 'grey'}]

## 4. Get Word Embedding & Sentence Embedding

### Tokenization

In [ ]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "the bird has very long and large brown wings, as well as a black body and a long black beak."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
the           1,996
bird          4,743
has           2,038
very          2,200
long          2,146
and           1,998
large         2,312
brown         2,829
wings         4,777
,             1,010
as            2,004
well          2,092
as            2,004
a             1,037
black         2,304
body          2,303
and           1,998
a             1,037
long          2,146
black         2,304
beak         23,525
.             1,012
[SEP]           102


### Segement ID

In [ ]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Extracting Embeddings

In [ ]:
# Convert input list to torch.tensor
import torch
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
# Load pretrained model
from transformers import BertModel
model2 = BertModel.from_pretrained(path_model, output_hidden_states=True) 
model2.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model2(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [ ]:
# Understanding the outputs
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 24
Number of hidden units: 768


### Word Embedding

Grouping the values by layer makes sense for the model, but for our purposes we want it grouped by token.

Current dimensions:

```[# layers, # batches, # tokens, # features]```

Desired dimensions:

```[# tokens, # layers, # features]```

In [ ]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 24, 768])


In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 24, 768])

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 24, 768])

In [ ]:
# Swap dimensions 0 and 1 (layers and tokens).
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()

torch.Size([24, 13, 768])

### Sentence Embedding
- a simple approach is to average the second to last hiden layer of each token
- 768 length vector

In [ ]:
sentence_embedding = torch.mean(hidden_states[-2][0], dim=0)

## References
- [huggingface examples - language modeling](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=VmaHZXzmkNtJ)  
- [BERT Fine-Tuning Tutorial with PyTorch, Chris McCormick and Nick Ryan](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#31-bert-tokenizer)